In [1]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy import sparse

df = pd.read_csv('/home/yuanzhi/Text_individual/codes/macro_news_2021_sp500.csv')
df.title=df.title.apply(lambda x: eval (x))

In [2]:
print('columns', df.columns)
week_news = df.title
pred_ret = df.pred_ret
pnn = 0
for i in range(len(week_news)):
    pnn += len(week_news[i])


columns Index(['week', 'title', 'DateEnd', 'contem_ret', 'pred_ret'], dtype='object')


In [23]:
pnn

23169

In [33]:
'''
Author: Naixin && naixinguo2-c@my.cityu.edu.hk
Date: 2023-01-02 14:18:49
LastEditors: Naixin && naixinguo2-c@my.cityu.edu.hk
LastEditTime: 2023-01-03 10:02:57
FilePath: /Gtext/pretrained/bert.ipynb
Description: 

'''
from huggingface_hub import notebook_login
notebook_login()
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import transformers
##hkust Contemporary Accounting Research 2022sep
#(i) corporate annual and quarterly filings—60,490 10-Ks and 142,622 10-Qs of Russell 3000 firms between 1994 and 2019 from the SEC’s EDGAR website; 
# (ii) financial analyst reports—476,633 reports issued for S&P 500 firms between 2003 and 2012 from the Thomson Investext database; 
# (iii) earnings conference call transcripts—136,578 transcripts of 7,740 public firms between 2004 and 2019 from the SeekingAlpha website
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert1 = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer1 = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp1 = pipeline("sentiment-analysis", model=finbert1, tokenizer=tokenizer1)
#amsterdam2019
#TRC24, which consists of 1.8M news articles that were published by Reuters between 2008 and 2010.
#FiQA Sentiment, which created for WWW ’18 conference financial opinion mining and question answering challenge
#Financial PhraseBank,consists of 4845 english sentences selected randomly from financial news found on LexisNexis database.These sentences then were annotated by 16 people with background in finance and business.
#

finbert2 = BertForSequenceClassification.from_pretrained('ProsusAI/finbert',num_labels=3)
tokenizer2 = BertTokenizer.from_pretrained('ProsusAI/finbert')

nlp2 = pipeline("sentiment-analysis", model=finbert2, tokenizer=tokenizer2)


from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer0 = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
finbert0 = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

nlp0 = pipeline("sentiment-analysis", model=finbert0, tokenizer=tokenizer0)

cpu


In [36]:
results1,results2,results0 = [],[],[]
for i in range(len(week_news)):
    results1.append(nlp1(week_news[i]))
    results2.append(nlp2(week_news[i]))
    results0.append(nlp0(week_news[i]))

In [ ]:
sentences1 = ["Analysis: Putin Prepares Russians for a Long Fight Ahead",  
             "Europe Futures Climb as Hong Kong Stocks Rally: Markets Wrap", 
             "Fed Minutes to Reveal Source of Inflation Angst Pushing Up Rates"]
results1 = nlp1(sentences1)
print(results1)  
results2 = nlp2(sentences1)
print(results2)  

results0 = nlp0(sentences1)
print(results0)  


In [20]:
len(results2)


23169

In [48]:
hkfin_neg,hkfin_pos ,hkscore= [],[],[]
for i in results1:
    nlp1neg,nlp1pos,score=[],[],[]
    
    for j in range(len(i)):
        score.append(i[j]['score'])
        if i[j]['label'] == 'Negative':
            nlp1neg.append(j)
            # results1[i]['score'] = -results1[i]['score']
        elif i[j]['label'] == 'Positive':
            nlp1pos.append(j)
    hkfin_neg.append(nlp1neg)
    hkfin_pos.append(nlp1pos)
    hkscore.append(score)
    
amsfin_neg,amsfin_pos,amsscore= [],[],[]
for i in results2:
    nlp2neg,nlp2pos,score=[],[],[]
    for j in range(len(i)):
        score.append(i[j]['score'])
        if i[j]['label'] == 'negative':
            nlp2neg.append(j)
            # results1[i]['score'] = -results1[i]['score']
        elif i[j]['label'] == 'positive':
            nlp2pos.append(j)
    amsfin_neg.append(nlp2neg)
    amsfin_pos.append(nlp2pos)
    amsscore.append(score)
distill_neg,distill_pos,distillscore = [],[],[]
for i in results0:
    nlp0neg,nlp0pos,score=[],[],[]
    for j in range(len(i)):
        score.append(i[j]['score'])
        if i[j]['label'] == 'NEGATIVE':
            nlp0neg.append(j)
            # results1[i]['score'] = -results1[i]['score']
        elif i[j]['label'] == 'POSITIVE':
            nlp0pos.append(j)
    distill_neg.append(nlp0neg)
    distill_pos.append(nlp0pos)
    distillscore.append(score)
            

In [38]:
print('num hkfin_pos:',sum(len(hkfin_pos[i]) for i in range(len(hkfin_pos))),'num hkfin_neg:',sum(len(hkfin_neg[i]) for i in range(len(hkfin_neg))),'num hkfin_natural:',pnn-sum(len(hkfin_pos[i]) for i in range(len(hkfin_pos)))-sum(len(hkfin_neg[i]) for i in range(len(hkfin_neg))))
print('num amsfin_pos:',sum(len(amsfin_pos[i]) for i in range(len(amsfin_pos))),'num amsfin_neg:',sum(len(amsfin_neg[i]) for i in range(len(amsfin_neg))),'num amsfin_natural:',pnn-sum(len(amsfin_pos[i]) for i in range(len(amsfin_pos)))-sum(len(amsfin_neg[i]) for i in range(len(amsfin_neg))))
print('num distill_pos:',sum(len(distill_pos[i]) for i in range(len(distill_pos))),'num distill_neg:',sum(len(distill_neg[i]) for i in range(len(distill_neg))),'num distill_natural:',pnn-sum(len(distill_pos[i]) for i in range(len(distill_pos)))-sum(len(distill_neg[i]) for i in range(len(distill_neg))))

num hkfin_pos: 2910 num hkfin_neg: 3928 num hkfin_natural: 16331
num amsfin_pos: 2593 num amsfin_neg: 5469 num amsfin_natural: 15107
num distill_pos: 10328 num distill_neg: 12841 num distill_natural: 0


In [91]:
print(sum(len(set(hkfin_pos[i]) &set(amsfin_pos[i])) for i in range(len(week_news))),sum(len(set(amsfin_pos[i])& set(distill_pos[i])) for i in range(len(week_news))),sum(len(set(hkfin_pos[i])& set(distill_pos[i])) for i in range(len(week_news))))
print(sum(len(set(hkfin_neg[i]) &set(amsfin_neg[i])) for i in range(len(week_news))),sum(len(set(amsfin_neg[i])& set(distill_neg[i])) for i in range(len(week_news))),sum(len(set(hkfin_neg[i])& set(distill_neg[i])) for i in range(len(week_news))))

1325 1634 2067
2417 4876 3351


In [123]:
import heapq
hkdistill_pos,hkdistill_neg, amsdistill_pos,amsdistill_neg, hkams_pos, hkams_neg=[],[],[],[],[],[]
hkscorepos,hkscoreneg,amsscorepos,amsscoreneg,distillscorepos,distillscoreneg =[],[],[],[],[],[]
for i in range(len(week_news)):
    hkscorepos +=list(np.array(hkscore[i])[hkfin_pos[i]])
    hkscoreneg += list(np.array(hkscore[i])[hkfin_neg[i]])
    
    amsscorepos += list(np.array(amsscore[i])[amsfin_pos[i]])
    amsscoreneg +=list(np.array(amsscore[i])[amsfin_neg[i]])
    
    distillscorepos +=list(np.array(distillscore[i])[distill_pos[i]])
    distillscoreneg +=list(np.array(distillscore[i])[distill_neg[i]])
    
hk1=[]
for i in set(heapq.nlargest(2000, hkscorepos)):
    hk1+=list(np.where(hkscorepos==i)[0])
hk0=[]
for i in set(heapq.nlargest(2000, hkscoreneg)):
    hk0+=list(np.where(hkscoreneg==i)[0])   
ams1=[]
for i in set(heapq.nlargest(2000, amsscorepos)):
    ams1+=list(np.where(amsscorepos==i)[0])
ams0=[]
for i in set(heapq.nlargest(2000, amsscoreneg)):
    ams0+=list(np.where(amsscoreneg==i)[0]) 
distill1=[]
for i in set(heapq.nlargest(2000, distillscorepos)):
    distill1+=list(np.where(distillscorepos==i)[0])
distill0=[]
for i in set(heapq.nlargest(2000, distillscoreneg)):
    distill0+=list(np.where(distillscoreneg==i)[0])      
hkams_pos=set(hk1)&set(ams1)
hkams_neg=set(hk0)&set(ams0)

hkdistill_pos=set(hk1)&set(distill1)
hkdistill_neg=set(hk0)&set(distill0)

amsdistill_pos=set(ams1)&set(distill1)
amsdistill_neg=set(ams0)&set(distill0)
# hkams_pos=set(map(hkscorepos.index, heapq.nlargest(1000, hkscorepos)))&set(map(amsscorepos.index, heapq.nlargest(1000, amsscorepos)))
# hkams_neg=set(map(hkscoreneg.index, heapq.nlargest(1000, hkscoreneg)))&set(map(amsscoreneg.index, heapq.nlargest(1000, amsscoreneg)))

# hkdistill_pos=set(map(hkscorepos.index, heapq.nlargest(1000, hkscorepos)))&set(map(distillscorepos.index, heapq.nlargest(1000, distillscorepos)))
# hkdistill_neg=set(map(hkscoreneg.index, heapq.nlargest(1000, hkscoreneg)))&set(map(distillscoreneg.index, heapq.nlargest(1000, distillscoreneg)))

# amsdistill_pos=set(map(amsscorepos.index, heapq.nlargest(1000, amsscorepos)))&set(map(distillscorepos.index, heapq.nlargest(1000, distillscorepos)))
# amsdistill_neg=set(map(amsscoreneg.index, heapq.nlargest(1000, amsscoreneg)))&set(map(distillscoreneg.index, heapq.nlargest(1000, distillscoreneg)))

print('hkams_pos:',len(hkams_pos),'hkams_neg:',len(hkams_neg))
print('amsdistill_pos:',len(amsdistill_pos),'amsdistill_neg:',len(amsdistill_neg))
print('hkdistill_pos:',len(hkdistill_pos),'hkdistill_neg:',len(hkdistill_neg))


hkams_pos: 1377 hkams_neg: 719
amsdistill_pos: 449 amsdistill_neg: 278
hkdistill_pos: 433 hkdistill_neg: 286


In [134]:
len(all)

2910

In [141]:
all = []
for i in range(len(week_news)):
    all+=list(np.array(week_news[i])[hkfin_pos[i]])
seesee=np.array(all)[hk1]
seesee[:10]

array(['Americans Deserve Better Than Clickbait Crack',
       "Workers' Commutes Expected to Improve",
       'Shipping Firm Bets Small Size Is Strength',
       'Boss Talk: Fast-Food CEO Orders Up Online Growth',
       "BlackRock's Profit And Assets Surge",
       'World News: China Exports Hit Record in 2020, Nation withstands pandemic to post best performance among top global economies',
       'Stellantis CEO Sees Strength in Brands, Chief of combined auto maker plans to sell vehicles under 14 lineups in 130 nations',
       "REVIEW, How Higher Wages Can Increase Profits, Many companies have cut or frozen employees' pay during the pandemic, but research shows that paying workers more can boost the bottom line by increasing productivity and retention.",
       'P&G Gets Shoppers to Pay Up, Strong sales increases reflect consumer shift to pricier products for cleaning and hygiene',
       'Better Than Piracy'], dtype='<U331')

In [140]:
all = []
for i in range(len(week_news)):
    all+=list(np.array(week_news[i])[hkfin_neg[i]])
seesee=np.array(all)[hk0]
seesee[:10]

array(['States Struggle To Get Vaccines Deployed, Logistical snags plague local officials after government sent doses, little guidance',
       "Business News: Ford's Earnings Take a Hit From Decreased Truck Output",
       'Gucci Revenue Sinks, Hurt by Fewer Tourists, Limited New Products',
       "Business News: Meat Processing Stalled By Cold, Cargill, Pilgrim's Pride, Tyson, others suspend some operations amid harsh wintery storms",
       "Business News: Nike's Sales Fall Short of Forecasts",
       'Banking Shares Drive Down S&P 500, Investors show concern about losses tied to fund unloading billions in holdings',
       'Supplier Misses Out on Grocery Revival, Maker of store-brand product lost share as supermarket chains gained during crisis',
       "World News: India's Virus Surge Hurts Vaccine Efforts, Exports slow from leading producer, putting more pressure on developing nations",
       'Pace of Hiring Falls Unexpectedly, Job growth is sharply lower than forecast as some s

In [145]:
all = []
for i in range(len(week_news)):
    all+=list(np.array(week_news[i])[amsfin_pos[i]])
seesee=np.array(all)[ams1]
seesee[:10]

array(['On-Time Deliveries Improve, for Most Part',
       'Business News: Publicis Revenue Rose in Quarter',
       'Business News: Zoom Video Sees Strong Growth, Company expects revenue to rise more than 41% even as pandemic eases',
       "Spotify Keeps Gaining Listeners, Provider of streaming content has another strong quarter, helped by 'Rogan' popularity",
       'Business News: Volvo Sees Steady Growth in EV Sales',
       'U.S. News: Social Security Benefits to Rise 5.9%',
       'Business News: Albertsons Sales Growth Outpaces Rivals in Pandemic',
       'Business News: EV Maker Aims for Improved Ratings',
       'Apollo Posts Higher Earnings as Value Of Portfolio Climbs',
       "World News: China's Exports Expanded In August"], dtype='<U331')

In [144]:
all = []
for i in range(len(week_news)):
    all+=list(np.array(week_news[i])[amsfin_neg[i]])
seesee=np.array(all)[ams0]
seesee[:10]

array(["Business News: Oscars Drew Its Smallest TV Audience, The ABC broadcast attracted 9.85 million viewers, down 58% from last year's event",
       'Pace of Hiring Falls Unexpectedly, Job growth is sharply lower than forecast as some sectors struggle to find enough workers',
       "Business News: Nike's Sales Fall Short of Forecasts",
       "EXCHANGE, Business News: Ford's June Sales Slid, Hurt by Chip Shortage",
       'Banking & Finance: AIG Income Falls Sharply on Covid Costs, Insurer reports $178 million in underwriting losses tied to event, travel cancellations',
       'Variant Slams Small-Cap Stocks, The Russell 2000 benchmark has dropped 7.4% since Omicron surfaced',
       'U.S. News: Jobless Claims Rise, Total Recipients Fall, Uptick in applications tied to auto industry; rolls are the lowest since late March 2020',
       'Business News: Container Imports Fall at California Ports',
       'Shopper Pullback Leads to 1.1% Drop In July Retail Sales',
       'Media & Techn